Asynchrone Microservices
------------------------

![](images/Microservices-Messaging.png)

Quelle: Buch Microservices Rezepte
- - -

Das System besteht aus einem Microservice **order**, der eine Bestellung über die Weboberfläche entgegennimmt. 

Die Bestellung schickt der Bestellprozess dann als Record über Kafka an den Microservice für den Versand **shipping** und den Microservice für die Erstellung der Rechnung **invoicing**.

Die Bestellung wird als JSON übertragen. So können der Rechnungs-Microservice und der Versand-Microservice aus der Datenstruktur jeweils die Daten auslesen, die für den jeweiligen Microservice relevant sind.

Der Versand-Microservice und der Rechnungs-Microservice speichern die Informationen aus den Records in ihren eigenen Datenbank-Schemata. Alle Microservices nutzen eine gemeinsame Postgres-Datenbank.



In [ ]:
%%bash
kubectl create namespace ms-kafka
kubectl label  namespace ms-kafka istio-injection=enabled
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/invoicing.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/kafka.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/postgres.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/shipping.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/zookeeper.yaml
kubectl get service/apache --namespace ms-kafka

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

Wird dieser URL in einem neuen Tab geöffnet, wird ein Menu angezeigt.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-kafka apache -o=jsonpath='{ .spec.ports[0].nodePort }')

- -  -
Nachdem wir ein paar Aufträge erfasst haben, können wir uns die Meldungen im Messaging Server (Kafka) anschauen.


In [ ]:
! kubectl exec -n ms-kafka  $(kubectl get po -l app=kafka -n ms-kafka -o=jsonpath='{ .items[0].metadata.name }') -- bash /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic order --from-beginning --timeout-ms 2000 

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-kafka